In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import year, month, to_date, date_format, lit
from pyspark import SparkConf
from botocore.exceptions import ClientError
import boto3
import botocore
import os
import awswrangler as wr

In [ ]:
# Get env vars
database_name = os.environ.get("DB_NAME")
table_name = os.environ.get("TB_NAME")
bucket_name = os.environ.get("BUCKET_NAME")
work_group = os.environ.get("WORKGROUP")
region = os.environ.get("AWS_REGION")

In [ ]:
# Create SparkSession
spark = SparkSession.builder \
    .appName('sample_spark_app') \
    .config("spark.sql.catalog.AwsGlueCatalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.AwsGlueCatalog.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config("spark.sql.catalog.AwsGlueCatalog.warehouse", "s3a://bd-datawarehouse/") \
    .config("spark.sql.catalog.AwsGlueCatalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .getOrCreate()

# Set log level to WARN
spark.sparkContext.setLogLevel("WARN")

# Enable schema evolution (Allows Iceberg to add new columns)
spark.conf.set("spark.sql.iceberg.schema.evolution.enabled", "true")


In [ ]:
# Read a sample csv dataset
df = spark.read.csv('data/tips.csv', inferSchema=True, header=True)

In [ ]:
# Its just a good practice to add a 'created_at' column storing data timestamp. It also will be used in this sample as Iceberg partion in the form: month('created_at')
# Create 'created_at' column and init
df = df.withColumn('created_at', F.current_timestamp())

# Correct way to reorder (and avoid duplicates):
df = df.select("created_at", *[col for col in df.columns if col != "created_at"])


In [ ]:
def create_s3_bucket_if_not_exists(bucket_name, region_name=None):
    try:
        s3 = boto3.client('s3', region_name=region_name)
        # Check if the bucket exists
        try:
            s3.head_bucket(Bucket=bucket_name)
            print(f"Bucket '{bucket_name}' already exists.")
            return True
        except ClientError as e:
        # except boto3.exceptions.ClientError as e:
            error_code = e.response['Error']['Code'] 
            if error_code == '404':
                print(f"Bucket '{bucket_name}' does not exist. Creating...")
                if region_name:
                    try:
                        s3.create_bucket(
                            Bucket=bucket_name,
                            CreateBucketConfiguration={'LocationConstraint': region_name}
                        )
                    except Exception as e:
                        print(f"Error creating bucket in region {region_name}: {e}")
                        return False
                else:
                    try:
                        s3.create_bucket(Bucket=bucket_name)
                    except Exception as e:
                        print(f"Error creating bucket: {e}")
                        return False
                print(f"Bucket '{bucket_name}' created successfully.")
                return True

            else:
                print(f"An unexpected error occurred checking or creating bucket: {e}")
                return False
    except Exception as e:
        print(f"A general error occurred: {e}")
        return False


create_s3_bucket_if_not_exists(bucket_name, region)

In [ ]:
# Add new column regarding vat taxes of 21% 
df = df.withColumn('vat', F.col('total_bill') * 0.21)

In [ ]:
# Define S3 bucket and path for database and table
s3_path = f"{database_name}/{table_name}/"  # Path inside the bucket

# Initialize S3 client
s3 = boto3.client("s3")

def s3_path_exists(bucket, path):
    """Check if an S3 path exists by listing objects with that prefix."""
    response = s3.list_objects_v2(Bucket=bucket, Prefix=path)
    return "Contents" in response  # Returns True if objects exist

def create_s3_path(bucket, path):
    """Create an empty directory in S3 by uploading an empty file."""
    if not s3_path_exists(bucket, path):
        s3.put_object(Bucket=bucket, Key=f"{path}placeholder.txt", Body=b"")  # Upload an empty file
        print(f"Created path: s3://{bucket}/{path}")
    else:
        print(f"Path already exists: s3://{bucket}/{path}")

# Check and create the path if needed
create_s3_path(bucket_name, s3_path)

In [ ]:
# Create database if not exists
glue_client = boto3.client('glue', region_name="eu-west-1")  # Change to your region

database_location = f"s3://{bucket_name}/{database_name}"

# Check if the database already exists
existing_databases = [db['Name'] for db in glue_client.get_databases()['DatabaseList']]
if database_name not in existing_databases:
    glue_client.create_database(DatabaseInput={'Name': database_name, 'LocationUri': database_location})
    print(f"Database {database_name} created successfully.")
else:
    print(f"Database {database_name} already exists.")

In [ ]:
# We have to options in order to partition by month
# Create a non hidden field in order to be used as partition field
# df = df.withColumn("yearmonth", date_format(F.col("created_at"), "yyyyMM").cast("int")) # or string
# or just use a hidden partition with transform method, e.g.: "PARTITIONED BY (month(created_at))", check https://medium.com/@life-is-short-so-enjoy-it/aws-athena-iceberg-experiment-dropping-partitions-month-b5074e56c911 in order to understand what partitions month value means.
# Iceberg table properties could be readed using: "describe formatted <database_name>.<table_name>;"


# Create Iceberg table if not exist partionted by created_at
spark.sql(f"""    
    CREATE TABLE IF NOT EXISTS AwsGlueCatalog.{database_name}.{table_name} (
        created_at timestamp
    )
    USING iceberg
    PARTITIONED BY (month(created_at))
    LOCATION 's3://{bucket_name}/{database_name}/{table_name}'
    TBLPROPERTIES (
        'format'='parquet',        
        'write_compression'='ZSTD',
        'write.spark.accept-any-schema'='true',
        'history.expire.min-snapshots-to-keep'='6',
        'history.expire.max-snapshot-age-ms'='432000000'        
    )
""")


# 'optimize_rewrite_delete_file_threshold'='1',
# 'optimize_rewrite_data_file_threshold'='3',
# 'write.spark.accept-any-schema'='true' 
# 'vacuum_max_snapshot_age_seconds'='10800',
# 'vacuum_min_snapshots_to_keep'='4',

In [ ]:
# Write to table using awswrangler.athena.to_iceberg -> https://aws-sdk-pandas.readthedocs.io/en/3.2.1/stubs/awswrangler.athena.to_iceberg.html
wr.athena.to_iceberg(
        df=df.toPandas(),
        database=database_name,
        table=table_name,   
        temp_path=f"s3://{bucket_name}/{database_name}/tmp_table_{table_name}/",
        workgroup=work_group,
        keep_files=False,
        schema_evolution=True,
        fill_missing_columns_in_df=True
    )

In [ ]:
df = spark.sql(f"DESCRIBE TABLE EXTENDED AwsGlueCatalog.{database_name}.{table_name}")
df.show(100, False)

In [ ]:
spark.stop()